In [1]:
using Polymake
using Test

polymake version 4.11
Copyright (c) 1997-2023
Ewgenij Gawrilow, Michael Joswig, and the polymake team
Technische Universität Berlin, Germany
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
p = polytope.Polytope(POINTS=[1 -1 -1; 1 1 -1; 1 -1 1; 1 1 1; 1 0 0])
@show p.VERTICES

p.VERTICES = pm::Matrix<pm::Rational>
1 -1 -1
1 1 -1
1 -1 1
1 1 1



polymake: used package ppl
  The Parma Polyhedra Library ([[wiki:external_software#PPL]]): A C++ library for convex polyhedra
  and other numerical abstractions.
  http://www.cs.unipr.it/ppl/



pm::Matrix<pm::Rational>
1 -1 -1
1 1 -1
1 -1 1
1 1 1


In [3]:
MAX_LATTICE_POINTS = 26 #global variable for the maximum number of lattice points
polytopes_list = [] #global list of polytopes

for i in 4:2:20
    open("SmoothGeneration_output/SmoothGeneration_output$i") do f #open the file
        number_polytopes = 0
        polytopes_read = 0
        while ! eof(f)
            #Read the number of vertices
            number_vertices = parse(Int, readline(f))
            polytopes_read += 1
            
            vertices_matrix = zeros(Int, number_vertices, 4)
            #Read the matrix of vertices of polytopes P
            for a in 1:number_vertices
                vertices_string = readline(f)
                vertex_vector = [1;parse.(Int64, split(vertices_string))] #append 1 to the beginning due to polymake standards
                vertices_matrix[a, :] .= vertex_vector
            end
            #Create the polytope from its vertices
            new_polytope = polytope.Polytope(POINTS=vertices_matrix)
            
            @assert new_polytope.SMOOTH #Checks if the polytope is smooth
            if new_polytope.N_LATTICE_POINTS <= MAX_LATTICE_POINTS #Check if the polytope does not exceed the maximum number of lattice points
                isomorphic_found = false
                #Check we polytopes_list already has this polytope
                for smooth_polytope in polytopes_list
                    if polytope.lattice_isomorphic_smooth_polytopes(smooth_polytope, new_polytope) #Check if any of the polytopes are lattice-equivalent
                        isomorphic_found = true #Raise flag
                        break
                    end
                end
                if !isomorphic_found
                    push!(polytopes_list, new_polytope) #Append to the list
                    number_polytopes += 1
                end
            end
        end 
        println(number_polytopes, " smooth polytopes extracted of ", polytopes_read, " polytopes read from file")
    end
end

3 smooth polytopes extracted of 3 polytopes read from file
293

polymake: used package bliss
  [[wiki:external_software#bliss]] is a software for computation of automorphism groups of graphs.
  Copyright by Tommi Junttila and Petteri Kaski.
  http://www.tcs.hut.fi/Software/bliss/


 smooth polytopes extracted of 1386 polytopes read from file
177 smooth polytopes extracted of 2858 polytopes read from file
54 smooth polytopes extracted of 664 polytopes read from file
45 smooth polytopes extracted of 1885 polytopes read from file
5 smooth polytopes extracted of 784 polytopes read from file
3 smooth polytopes extracted of 791 polytopes read from file
0 smooth polytopes extracted of 257 polytopes read from file
0 smooth polytopes extracted of 115 polytopes read from file


In [10]:
#Outputting the Smooth 3-polytopes after creating the list
output_file = "prune_output" #filename

if !isfile(output_file)
    touch(output_file)  # Create an empty file if it doesn't exist
end

for smooth_polytope in polytopes_list
    @assert smooth_polytope.SMOOTH
    @assert smooth_polytope.NORMAL
    
    open(output_file, "a") do file #Open the file in appending mode
        println(file, smooth_polytope.N_VERTICES)

        #Printing the contents of the matrix and avoiding the Polymake types showing up
        for row in eachrow(smooth_polytope.VERTICES) 
            println(file, join(row[2:end], " "))
        end
    end
end

polymake: used package 4ti2
  4ti2 -- A software package for algebraic, geometric and combinatorial problems on linear spaces.
  Copyright by 4ti2 team.
  http://www.4ti2.de/



In [4]:
#Performing various analyses on the produced polytopes

for smooth_polytope in polytopes_list
    if smooth_polytope.N_INTERIOR_LATTICE_POINTS >= 1
        println(smooth_polytope.N_INTERIOR_LATTICE_POINTS)
    end
end

println(size(polytopes_list))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
(580,)


In [24]:
p = polytopes_list[569]
@show p.VERTICES
println(p.INTERIOR_LATTICE_POINTS)
p.VISUAL

p.VERTICES = pm::Matrix<pm::Rational>
1 0 0 0
1 0 1 0
1 2 0 0
1 2 1 0
1 0 0 2
1 0 3 2
1 0 3 3
1 0 1 3
1 2 0 1
1 1 0 2
1 2 2 1
1 1 3 2

pm::Matrix<pm::Integer>
1 1 1 1



LoadError: polymake: unknown property Polytope<Rational>::VISUAL at /home/kyle/.julia/scratchspaces/d720cf60-89b5-51f5-aff5-213f193123e7/polymake_12794641098094048062_1.8_depstree_v2/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 432.
	Polymake::Core::BigObjectType::property(Polymake::Core::BigObjectType=ARRAY(0x891dc70), "VISUAL") called at /home/kyle/.julia/scratchspaces/d720cf60-89b5-51f5-aff5-213f193123e7/polymake_12794641098094048062_1.8_depstree_v2/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 714
	Polymake::Core::BigObjectType::encode_descending_path(Polymake::Core::BigObjectType=ARRAY(0x891dc70), "VISUAL") called at /home/kyle/.julia/scratchspaces/d720cf60-89b5-51f5-aff5-213f193123e7/polymake_12794641098094048062_1.8_depstree_v2/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 757
	Polymake::Core::BigObjectType::encode_read_request(Polymake::Core::BigObjectType=ARRAY(0x891dc70), "VISUAL") called at /home/kyle/.julia/scratchspaces/d720cf60-89b5